In [ ]:
import glob
import numpy as np

import scipy.io as sio

%load_ext autoreload
%autoreload 1

In [ ]:
from src.feat_extraction import bandpower_multitaper

In [ ]:
no_sessions = 3
no_participants = 15
no_videos = 15
no_channels = 62
frequency = 200

## Create features

In [ ]:
files_session = []
files_session = glob.glob("./data/dataset/SEED/Preprocessed_EEG/*.mat")

print(np.shape(files_session))

files_session = sorted(files_session)
files_session = np.concatenate((files_session[6*no_sessions:], files_session[:6*no_sessions]))

#### Multitaper

In [ ]:
def search(myDict, lookup):
    for key, value in myDict.items():
        if str.find(key, lookup) != -1:
            return(key)

bandpower_SEED_welch = []
for i in range(no_sessions*no_participants):
    mat = sio.loadmat(files_session[i], verify_compressed_data_integrity=False)
    for j in range(no_videos):
        key = search(mat, '_eeg'+str(j+1))
        input_brainwaves = mat[key]
        input_brainwaves = np.array(input_brainwaves)
        bands_video = []
        for k in range(no_channels):
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[4, 7], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[8, 13], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[14, 30], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[31, 50], relative=False))
               
        bandpower_SEED_welch.append(bands_video)
    print(i, np.shape(bandpower_SEED_welch))
    np.save('./data/bandpower_SEED_multitaper', np.array(bandpower_SEED_welch))

bandpower_SEED_welch = np.array(bandpower_SEED_welch)
print(bandpower_SEED_welch.shape)
np.save('./data/bandpower_SEED_multitaper', bandpower_SEED_welch)